In [ ]:
import json
from twitter import *
import time
import pandas as pd
import os


In [ ]:

#-----------------------------------------------------------------------
# load our API credentials  from the config.py file
#-----------------------------------------------------------------------
config = {}
exec(open('config.py').read(), config)

#-----------------------------------------------------------------------
# create twitter API object
#-----------------------------------------------------------------------
twitter = Twitter(auth = OAuth(config["access_key"], config["access_secret"], config["consumer_key"], config["consumer_secret"]))




In [ ]:
#function to parse twitter JSON
#json.loads : Parse JSON - Convert from JSON to Python
#json.dumps : Convert from Python to JSON

def pp_json(json_thing, sort=True, indents=4):
    if type(json_thing) is str:
        print(json.dumps(json.loads(json_thing), sort_keys=sort, indent=indents))
    else:
        print(json.dumps(json_thing, sort_keys=sort, indent=indents))
    return None

In [ ]:
# create data folder if it does not exists
if not 'data' in os.listdir():
    os.mkdir('data')
    
#####################################################################
### MAKE EMPTY DF To POPULATE WITH DATA ###########

df = pd.DataFrame(columns = ['created_at', 'tweet_id', 'text', 'url', 'retweet_count', 'status'])
df.to_csv(file_path)


In the next cell, set the earliest date of tweets to be grabbed.  So that all posts of that particular day is grabbed. However,  twitter api only allows around 3200 posts to be collected for one particular account. Hence, loop is designed to break once that number is reached which, depending on your settings, could happen before the earliest date is reached.

In [ ]:
earliest_date = 'Jan 17' #Set it to one day before the date you want.
screen_name = 'nytimes' #twitter screen name 
file_path = 'data/nytimesposts.csv' #file path for the csv file that will store data

In [ ]:

#-----------------------------------------------------------------------
# loop through each of newsoutlet statuses, and print its content
#-----------------------------------------------------------------------
count = 0

posts_collected = 0

# this is for breaking the loop if/when earliest_data is reached. 
# _continue is set to false when earliest_date is reached on line 58
_continue = True 

# every request gets you 200 tweets. So for each consecutive requests after the first one, we need to track 
# the id of the last tweet grabbed. So the next request grabs tweets before the last id.  

last_id = 0

while True:
 # Make API CALL
    try:
        
        if last_id == 0:
            count +=1
            print('request', count)
            data = twitter.statuses.user_timeline(screen_name=screen_name, count=200)
        else:
            # checking if the ealiest date has been reached or not.
            if _continue == False:
                print (f'DONE! All posts collected till {earliest_date}, 2019')
                break

            count +=1
            print('request', count)
            print('lastid', last_id)
            data = twitter.statuses.user_timeline(screen_name=screen_name, max_id = last_id-1, count=200)

        if len(data) == 0:
            #twitter API only allows around 3200 tweets to be collected for one particular user. 
            # After maxing out it returns an empty list.
            print (f'Maximum number of posts for one user reached, total no. of posts collected: {posts_collected}')
            break

        last_id = data[-1]['id']

    except Exception as E:
        print(E)
        # Max requests per 15 minutes is 75. The following will print out the message from twitter server 
        pp_json(twitter.application.rate_limit_status()["resources"]["statuses"]["/statuses/user_timeline"])
        print ("sleeping...")
        time.sleep(60*15)

    #Check how many posts grabbed
    posts_collected += len(data)
    print('Total posts collected:', posts_collected)

    # Loop through the posts to get information and populate df
    for status in data:
        if earliest_date in status['created_at']:
            _continue = False
            break

        mini_df = pd.DataFrame()
        mini_df['created_at'] = [status['created_at']]
        mini_df['tweet_id'] = [status['id']]
        mini_df['text'] = [status['text']]
        mini_df['url'] =  [status['user']['entities']['description']['urls'][0]['expanded_url']]
        mini_df['retweet_count'] = [status['retweet_count']]
        mini_df['status'] = [json.dumps(status)]
        mini_df.to_csv(file_path, mode='a', header=False)


# Get Retweeter List 

To collect list of retweeter id for each tweets collected above. 

In [ ]:

df= pd.read_csv('data/nytimesposts.csv')


### MAKE EMPTY DF To POPULATE WITH DATA 
file_path = 'data/retweeterlist.csv'
new_path = 'data/nytimesposts1.csv'

df1 = pd.DataFrame(columns = ['tweet_id', 'retweeter_list'])
df1.to_csv(file_path)

for i in range(df.shape[0]):
    
    try:
        mini_df = pd.DataFrame()
        r = twitter.statuses.retweeters.ids( _id=df.iloc[i, 2])['ids']
        mini_df['tweet_id'] = [df.iloc[i, 2]]
        mini_df['retweeter_list'] = [r]
        mini_df.to_csv(file_path, mode='a', header=False)
        print(f'index:{i}',  f'retweeter count:{len(r)}', 'original retweeter count:', df.iloc[i, 5])
    except Exception as e:
        print(e)
        print ("sleeping...")
        pp_json(twitter.application.rate_limit_status()["resources"]["statuses"]["/statuses/retweeters/ids"])
        time.sleep(60*15)

df1 = pd.read_csv(file_path)

if df.shape[0] == df1.shape[0]:
    df['retweeter_list'] = df1['retweeter_list']
    df.to_csv(new_path)
else:
    print('length of df and df1 do not match')


This whole process for entier 3200 is over 14 hours. So, to save time while this runs  I also run the get_user_info.py file in terminal to simultaneously get the individual user information as the retweeterlist.csv udpates. 